In [1]:
contest_folder = 'round2'   # CHANGE THIS

In [2]:
from analysis import get_current_status, select_solution

In [3]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [4]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [5]:
# df_grouped

In [6]:
aggregated_df

,problem_code,problem_name,statement,sample_in,sample_out,full_in,solution_id,execution_response,execution_code,execution_sample_out,execution_full_out,case_string,case_string_found,sample_out_found,hash,status,timestring
0,cottontail_climb_part_2,Cottontail Climb (Part 2),*This problem shares some similarities with pr...,6\n121 121 11\n0 100 2\n0 132 1\n121 132 1\n12...,Case #1: 1\nCase #2: 4\nCase #3: 12\nCase #4: ...,136\n121 121 11\n0 100 2\n0 132 1\n121 132 1\n...,"[001, 010]","[To solve this problem, we will generate all p...",[def read_int():\n return int(input())\n\nd...,[Case #1: 1\nCase #2: 4\nCase #3: 12\nCase #4:...,[Case #1: 1\nCase #2: 4\nCase #3: 12\nCase #4:...,"[Case #136, Case #136]","[True, True]","[False, False]",4cc1d9b5,not_analyzed,3130


In [ ]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=select_solution, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Running analysis at 3130
Running analysis at 3130 for Cottontail Climb (Part 2)
Selected solution 001 for cottontail_climb_part_2
Running analysis at 3200
Running analysis at 3230
Running analysis at 3300
Running analysis at 3300 for Cottontail Climb (Part 2)
Selected solution 002 for cottontail_climb_part_2
Running analysis at 3330
Running analysis at 3400
Running analysis at 3400 for Cottontail Climb (Part 2)
Selected solution 004 for cottontail_climb_part_2
Running analysis at 3430
Running analysis at 3500
Running analysis at 3530
Running analysis at 3600
Running analysis at 3630
